In [ ]:
!pip3 install yfinance

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import yfinance as yf

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
test = yf.Ticker("MSFT")